In [4]:
import os
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

In [ ]:
def extract_data(driver):
    try:
        app_rows = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.XPATH, "//div[@class='appRow']"))
        )
    except Exception as e:
        print(f"Error finding app rows: {e}")
        return [], [], [], [], []

    app_titles, developers, upload_dates, app_links, apk_links = [], [], [], [], []

    for app_row in app_rows:
        try:
            # Extract relevant data
            app_title = app_row.find_element(By.CLASS_NAME, 'appRowTitle').text
            app_link = app_row.find_element(By.CLASS_NAME, 'appRowTitle').find_element(By.TAG_NAME, 'a').get_attribute('href')
            developer = app_row.find_element(By.CLASS_NAME, 'byDeveloper').text
            upload_date = app_row.find_element(By.CLASS_NAME, 'dateyear_utc').text
            apk_link = app_row.find_element(By.CLASS_NAME, 'downloadLink').get_attribute('href')

            app_titles.append(app_title)
            developers.append(developer)
            upload_dates.append(upload_date)
            app_links.append(app_link)
            apk_links.append(apk_link)
        except Exception as e:
            print(f"Error extracting data for an app: {e}")

    return app_titles, developers, upload_dates, app_links, apk_links

In [18]:
url = 'https://www.apkmirror.com/?post_type=app_release&searchtype=apk&s=famapp'

# Configure the WebDriver and load the URL
try:
    driver = webdriver.Chrome()
    driver.get(url)
    print("URL loaded")
except Exception as e:
    print(f"Error loading URL: {e}")

# wait for the page to load
time.sleep(5)

app_titles, developers, upload_dates, app_links, apk_links = [], [], [], [], []

while True:
    # Extract data from the current page
    titles, devs, dates, links, apk_links_page = extract_data(driver)
    app_titles.extend(titles)
    developers.extend(devs)
    upload_dates.extend(dates)
    app_links.extend(links)
    apk_links.extend(apk_links_page)

    # Check if there's a "next" button to go to the next page
    try:
        next_page = driver.find_element(By.CLASS_NAME, "nextpostslink")
        next_page_url = next_page.get_attribute("href")
        driver.get(next_page_url)
    except Exception as e:
        print("No more pages found")
        break

# Save data to a dataframe
data = {'App Title': app_titles,
        'Developer': developers,
        'Upload Date': upload_dates,
        'App Link': app_links,
        'APK Link': apk_links}

df = pd.DataFrame(data)

driver.quit()

URL loaded
Error extracting data for an app: Message: no such element: Unable to locate element: {"method":"css selector","selector":".byDeveloper"}
  (Session info: chrome=112.0.5615.138)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00E1DCE3+50899]
	(No symbol) [0x00DAE111]
	(No symbol) [0x00CB5588]
	(No symbol) [0x00CE08F9]
	(No symbol) [0x00CE0AFB]
	(No symbol) [0x00CD8F71]
	(No symbol) [0x00CFB944]
	(No symbol) [0x00CD8EC6]
	(No symbol) [0x00CFBC54]
	(No symbol) [0x00D0E01C]
	(No symbol) [0x00CFB6F6]
	(No symbol) [0x00CD7708]
	(No symbol) [0x00CD886D]
	GetHandleVerifier [0x01083EAE+2566302]
	GetHandleVerifier [0x010B92B1+2784417]
	GetHandleVerifier [0x010B327C+2759788]
	GetHandleVerifier [0x00EB5740+672048]
	(No symbol) [0x00DB8872]
	(No symbol) [0x00DB41C8]
	(No symbol) [0x00DB42AB]
	(No symbol) [0x00DA71B7]
	BaseThreadInitThunk [0x75757EC9+25]
	RtlInitializeExceptionChain [0x7796D22B+107]
	RtlGetAppContainerNamedObjectPath [0x7796D1AD+237]

Error extracting data for an app: Messa

In [1]:
df.to_csv('apkmirror_fam.csv', index=False)

NameError: name 'df' is not defined

In [2]:
import re

def convert_url(input_url):
    # define the pattern to match the version number
    pattern = r'(\d+-){2}\d+'

    # define the base URL and the version string
    base_url = 'https://www.apkmirror.com/apk/fampay/fampay-prepaid-card-payments-for-teenagers/'
    version_string = 'fampay-teen-upi-payment-app-'

    # extract the version number from the input URL
    version_number = re.search(pattern, input_url).group(0)

    # construct the output URL
    output_url = base_url + version_number + '-release/' + version_string + version_number + '-android-apk-download/'

    # return the output URL
    return output_url


In [5]:
# open the csv file and save it to a dataframe

# in the dataframe, replace the APK link with the new URL using the convert_url function

df = pd.read_csv('apkmirror_fam.csv')
df['APK Link'] = df['APK Link'].apply(convert_url)
df.to_csv('apkmirror_fam.csv', index=False)